In [1]:
import pandas as pd
import pickle
import preprocess_utils

from_database = False

if from_database:
    # Load tables from database 
    dfs = preprocess_utils.load_all_tables()
else:
    # Open the file in binary read mode and load the pickle data
    with open('preprocessed_data/dfs.pickle', 'rb') as f:
        dfs = pickle.load(f)
        for key, df in dfs.items():
            print(f"DataFrame {key:<20} has {len(df):<6} rows and {len(df.columns):<6} columns.")

DataFrame abuse_report_types   has 4      rows and 4      columns.
DataFrame abuse_reports        has 0      rows and 8      columns.
DataFrame ar_internal_metadata has 1      rows and 4      columns.
DataFrame articles             has 400    rows and 33     columns.
DataFrame assertion_types      has 10     rows and 11     columns.
DataFrame assertion_versions   has 19668  rows and 10     columns.
DataFrame assertions           has 13941  rows and 14     columns.
DataFrame assertions_genes     has 2053   rows and 2      columns.
DataFrame assertions_tags      has 403    rows and 2      columns.
DataFrame assertions_technique_types has 0      rows and 2      columns.
DataFrame assertions_techniques has 0      rows and 2      columns.
DataFrame assessment_types     has 13     rows and 9      columns.
DataFrame assessments          has 0      rows and 6      columns.
DataFrame authors              has 797    rows and 10     columns.
DataFrame banned_orcid_users   has 0      rows and 5   

In [31]:
if from_database:
    # Save tables to pickle
    with open('preprocessed_data/dfs.pickle', 'wb') as f:
        pickle.dump(dfs, f)

    # load it back with:
    with open('preprocessed_data/dfs.pickle', 'rb') as f:
        dfs = pickle.load(f)


In [32]:
def clean_df(df):
    # Create a copy of the DataFrame at the start
    df = df.copy()
    
    columns_to_remove = ['user_id', 'orcid_user_id', 
                    'created_at', 'updated_at', 'assertion_updated_at', 
                    'workspace_id', 'user_id', 'doi', 'organism_id', # 'pmid', 
                    'all_tags_json', 'obsolete', 'ext', 'badge_classes','pluralize_title',
                    'can_attach_file', 'refresh_side_panel', 'icon_classes', 'btn_classes']
    patterns_to_remove = ['validated', 'filename', 'obsolete_article']
    
    # Remove existing columns
    existing_cols = [col for col in columns_to_remove if col in df.columns]
    if existing_cols:
        df = df.drop(existing_cols, axis=1)
    
    # Remove pattern-matched columns
    pattern_cols = []
    for pattern in patterns_to_remove:
        pattern_cols.extend([c for c in df.columns if pattern in c])
    if pattern_cols:
        df = df.drop(pattern_cols, axis=1)
    
    return df

def truncate_string(s, max_length=20):
    """Truncate string to max_length characters."""
    if isinstance(s, str) and len(s) > max_length:
        return s[:max_length] + '...'
    return s

 ## 2. Process articles from the database

In [73]:
# Process articles
articles = clean_df(dfs["articles"])
articles = articles.rename(columns={"id": "article_id"})
# add journals to articles:
journals = clean_df(dfs["journals"])
journals = journals.drop('tag', axis=1).rename(columns={"id": "journal_id", "name": "journal_name"})
articles = articles.merge(journals, on="journal_id", how="left", suffixes=('', '_journal')).drop("journal_id", axis=1)


# Extract first and last authors from authors_txt
def extract_authors(authors_txt):
    if isinstance(authors_txt, str) and authors_txt:
        authors_list = authors_txt.split(';')
        first_author = authors_list[0].strip() if authors_list else None
        last_author = authors_list[-1].strip() if authors_list else None
        return pd.Series([first_author, last_author])
    return pd.Series([None, None])

# Apply the function to create new columns
articles[['first_author_extracted', 'last_author_extracted']] = articles['authors_txt'].apply(extract_authors)

# Check the results
print(f"Total articles: {len(articles)}")
print(f"Articles with first author: {articles['first_author_extracted'].notna().sum()}")
print(f"Articles with last author: {articles['last_author_extracted'].notna().sum()}")

articles = articles.drop(columns=["key", "references_txt", "additional_context", "num",  "nber_panels", "large_scale", "nber_tables", "published_at"])


# Display the first few rows to verify
articles[['authors_txt', 'first_author_extracted', 'last_author_extracted']].head()

Total articles: 400
Articles with first author: 400
Articles with last author: 400


,authors_txt,first_author_extracted,last_author_extracted
0,Rizki RM;Rizki TM,Rizki RM,Rizki TM
1,Leulier F;Lhocine N;Lemaitre B;Meier P,Leulier F,Meier P
2,Kim YS;Han SJ;Ryu JH;Choi KH;Hong YS;Chung YH;...,Kim YS,Lee WJ
3,Tingvall TO;Roos E;Engström Y,Tingvall TO,Engström Y
4,Bhaskar V;Valentine SA;Courey AJ,Bhaskar V,Courey AJ


In [74]:
import json

# We need to analyze and clean up affiliations in the articles dataframe

def extract_primary_affiliation(affs_json_str):
    """Extract the first affiliation from the JSON string, safely handle bad JSON format."""
    try:
        # Parse the JSON string to a Python object
        affs_list = json.loads(affs_json_str)
        
        # Get the first non-empty affiliation
        for aff in affs_list:
            if aff and isinstance(aff, list) and len(aff) > 0 and aff[0].strip():
                return aff[0].strip()
        
        return None  # No valid affiliation found
    except (json.JSONDecodeError, TypeError, IndexError):
        # Handle poorly formatted JSON or other errors
        return None

# Create a new column with clean affiliations
articles['primary_affiliation'] = articles['affs_json'].apply(extract_primary_affiliation)

# Count articles with multiple vs. single affiliations
def count_affiliations(affs_json_str):
    """Count the number of non-empty affiliations in the JSON string."""
    try:
        affs_list = json.loads(affs_json_str)
        count = sum(1 for aff in affs_list if aff and isinstance(aff, list) and len(aff) > 0 and aff[0].strip())
        return count
    except (json.JSONDecodeError, TypeError, IndexError):
        return 0

articles['affiliation_count'] = articles['affs_json'].apply(count_affiliations)

# Identify articles with multiple affiliations
multiple_aff_articles = articles[articles['affiliation_count'] > 1]

# Print summary statistics
print(f"Total articles: {len(articles)}")
print(f"Articles with 1 affiliation: {len(articles[articles['affiliation_count'] == 1])}")
print(f"Articles with multiple affiliations: {len(multiple_aff_articles)}")
print(f"Articles with no valid affiliations: {len(articles[articles['affiliation_count'] == 0])}")

# Sample of articles with multiple affiliations
if not multiple_aff_articles.empty:
    print("\nSample of articles with multiple affiliations:")
    sample = multiple_aff_articles.head(5)
    for _, row in sample.iterrows():
        print(f"Article ID: {row['article_id']}, Title: {row['title'][:50]}...")
        try:
            affs = json.loads(row['affs_json'])
            for i, aff in enumerate(affs):
                if aff and isinstance(aff, list) and len(aff) > 0 and aff[0].strip():
                    print(f"  Affiliation {i+1}: {aff[0]}")
        except:
            print("  Error parsing affiliations")
        print()

Total articles: 400
Articles with 1 affiliation: 387
Articles with multiple affiliations: 0
Articles with no valid affiliations: 13


In [75]:
articles = articles.drop(columns=["affs_json", "affiliation_count"])

In [77]:
articles.to_csv("preprocessed_data/articles_db.csv", index=False)

 ## 2. Process claims from the database

In [78]:
# Main processing
claims = dfs["assertions"].copy()
print(len(claims))
claims = claims[claims["obsolete"] == False].copy()
print(len(claims))
claims = clean_df(claims)

id_cols = [col for col in claims.columns if "_id" in col]
print(id_cols)

claims = claims.merge(articles, on="article_id", how="left", suffixes=('', '_article'))
id_cols = [col for col in claims.columns if "_id" in col]
print(id_cols)


# Process assertion types
assertion_types = clean_df(dfs["assertion_types"])
assertion_types = assertion_types.rename(columns={"id": "assertion_type_id", "name": "assertion_type"})
claims = claims.merge(assertion_types, on="assertion_type_id", how="left", suffixes=('', '_assertion_type')).drop("assertion_type_id", axis=1)

# Process assessment types
assessment_types = clean_df(dfs["assessment_types"])
assessment_types = assessment_types.rename(columns={"id": "assessment_type_id", "name": "assessment_type"})
claims = claims.merge(assessment_types, on="assessment_type_id", how="left", suffixes=('', '_assessment_type')).drop("assessment_type_id", axis=1)

id_cols = [col for col in claims.columns if "_id" in col]
print(id_cols)

13941
13299
['article_id', 'assertion_type_id', 'assessment_type_id']
['article_id', 'assertion_type_id', 'assessment_type_id']
['article_id']


In [80]:
claims = claims.drop(['badge_tag_classes','description'], axis=1) # most not consistently used accross dataset
claims = claims.set_index('id')

In [82]:
# Update impact factor for "Proceedings. Biological sciences" and standardize the journal name
claims.loc[claims["journal_name"] == "Proceedings. Biological sciences", "impact_factor"] = 4.7 
claims.loc[claims["journal_name"] == "Proceedings. Biological sciences", "journal_name"] = "Proceedings Biological Sciences"

string_columns = claims.select_dtypes(include='object').columns
print(string_columns)
for col in string_columns:
    claims[col] = claims[col].apply(lambda x: x.replace('&amp;', '&') if isinstance(x, str) else x)

Index(['content', 'pmid', 'authors_txt', 'title', 'volume', 'issue',
       'abstract', 'journal_name', 'first_author_extracted',
       'last_author_extracted', 'primary_affiliation', 'assertion_type',
       'label', 'assessment_type'],
      dtype='object')


In [83]:

claims.to_csv('preprocessed_data/claims_db.csv')


string_columns = string_columns.drop(["assessment_type", "journal_name"])
df_truncated = claims.copy()
for col in string_columns:
    if col in df_truncated.columns:
        df_truncated[col] = df_truncated[col].apply(lambda x: truncate_string(x))

# Save truncated dataframe
df_truncated.to_csv('preprocessed_data/claims_db_truncated_for_llm.csv', index=False)
df_truncated

,content,article_id,rank,pmid,authors_txt,title,pmid_article,volume,issue,abstract,...,journal_name,impact_factor,first_author_extracted,last_author_extracted,primary_affiliation,assertion_type,label,is_assessed,assessment_type,rank_assessment_type
id,,,,,,,,,,,,,,,,,,,,,
5838,"<p>Belozerov VE, Lin...",2049.0,1,None,Zhuang ZH;Zhou Y;Yu ...,Regulation of Drosop...,16014325.0,18,4,The p38 mitogen-acti...,...,Cellular signalling,4.8,Zhuang ZH,Ge BX,Signal Transduction ...,reference,Reference,False,NaN,NaN
693,,2517.0,1,None,Choe KM;Werner T;Stö...,Requirement for a pe...,11872802.0,296,5566,Components of microb...,...,"Science (New York, N.Y.)",56.9,Choe KM,Anderson KV,Molecular Biology Pr...,assessment,Assessment,False,Not assessed,13.0
695,,2517.0,1,None,Choe KM;Werner T;Stö...,Requirement for a pe...,11872802.0,296,5566,Components of microb...,...,"Science (New York, N.Y.)",56.9,Choe KM,Anderson KV,Molecular Biology Pr...,assessment,Assessment,False,Not assessed,13.0
5840,Verified by many sub...,2204.0,1,None,Zettervall CJ;Anderl...,A directed screen fo...,15381778.0,101,39,An attack by a paras...,...,Proceedings of the National Academy of Science...,11.1,Zettervall CJ,Hultmark D,Umeå Centre for Mole...,assessment,Assessment,False,Verified,1.0
5839,Activation of a cell...,2204.0,1,None,Zettervall CJ;Anderl...,A directed screen fo...,15381778.0,101,39,An attack by a paras...,...,Proceedings of the National Academy of Science...,11.1,Zettervall CJ,Hultmark D,Umeå Centre for Mole...,main_claim,Main claim,True,Verified,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14468,The DIF antiserum us...,2898.0,1,None,Williams MJ;Rodrigue...,The 18-wheeler mutat...,9321392.0,16,20,Mammals and insects ...,...,The EMBO journal,11.4,Williams MJ,Eldon ED,Department of Biolog...,comment,Comment,False,Not assessed,13.0
11330,Although independent...,1577.0,1,None,Shia AK;Glittenberg ...,Toll-dependent antim...,19934223.0,122,Pt 24,"In Drosophila, the h...",...,Journal of cell science,4.0,Shia AK,Ligoxygakis P,Department of Bioche...,assessment,Assessment,False,"Unchallenged, logically consistent",6.0
11329,Spatzle (#gene:FBgn0...,1577.0,1,None,Shia AK;Glittenberg ...,Toll-dependent antim...,19934223.0,122,Pt 24,"In Drosophila, the h...",...,Journal of cell science,4.0,Shia AK,Ligoxygakis P,Department of Bioche...,main_claim,Main claim,True,"Unchallenged, logically consistent",6.0


## 3. Process authors from database

In [38]:
a = clean_df(dfs["authors"])  
a[a["article_id"] == 2053]

,id,article_id,name,sex,career_stage_id,leading_author,first_author,expertise_level_id
0,2,2053,Brun S,1,3,False,True,2
1,3,2053,Lemaitre B,1,4,True,False,2


In [85]:
a[a["name"] == "Lemaitre B"].head()

,id,article_id,name,sex,career_stage_id,leading_author,first_author,expertise_level_id
1,3,2053,Lemaitre B,1,4,True,False,2
7,8,3437,Lemaitre B,1,4,True,False,2
14,16,2801,Lemaitre B,1,4,True,False,2
30,36,2617,Lemaitre B,1,4,True,False,2
44,56,2869,Lemaitre B,1,4,True,False,2


In [28]:
a[a["article_id"] == 2451]

,id,article_id,name,sex,career_stage_id,leading_author,first_author,expertise_level_id
17,19,2451,De Gregorio E,1,3,False,True,2
95,20,2451,Lemaitre B,1,5,False,False,2
789,795,2451,Brey PT,1,5,True,False,2


In [86]:
articles

,article_id,authors_txt,title,pmid,volume,issue,abstract,year,journal_name,impact_factor,first_author_extracted,last_author_extracted,primary_affiliation
0,3347,Rizki RM;Rizki TM,Cell interactions in the differentiation of a ...,111992,12,3,The cellular events in the formation of melano...,1979,Differentiation; research in biological diversity,3.39,Rizki RM,Rizki TM,None
1,1987,Leulier F;Lhocine N;Lemaitre B;Meier P,The Drosophila inhibitor of apoptosis protein ...,16894030,26,21,The founding member of the inhibitor of apopto...,2006,Molecular and cellular biology,5.30,Leulier F,Meier P,The Breakthrough Toby Robins Breast Cancer Res...
2,2761,Kim YS;Han SJ;Ryu JH;Choi KH;Hong YS;Chung YH;...,"Lipopolysaccharide-activated kinase, an essent...",10636911,275,3,Eukaryotic organisms use a similar Rel/NF-kapp...,2000,The Journal of biological chemistry,4.80,Kim YS,Lee WJ,"Laboratory of Immunology, Medical Research Cen..."
3,2649,Tingvall TO;Roos E;Engström Y,The GATA factor Serpent is required for the on...,11274409,98,7,Innate immunity in Drosophila is characterized...,2001,Proceedings of the National Academy of Science...,11.10,Tingvall TO,Engström Y,Department of Molecular Biology and Functional...
4,3441,Bhaskar V;Valentine SA;Courey AJ,A functional interaction between dorsal and co...,10660560,275,6,To identify proteins that regulate the functio...,2000,The Journal of biological chemistry,4.80,Bhaskar V,Courey AJ,"Department of Chemistry and Biochemistry, Univ..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...
395,1882,Derré I;Pypaert M;Dautry-Varsat A;Agaisse H,RNAi screen in Drosophila cells reveals the in...,17967059,3,10,Chlamydia spp. are intracellular obligate bact...,2007,PLoS pathogens,6.70,Derré I,Agaisse H,"Section of Microbial Pathogenesis, Yale Univer..."
396,1815,Schmidt RL;Trejo TR;Plummer TB;Platt JL;Tang AH,Infection-induced proteolysis of PGRP-LC contr...,18308747,22,3,The Drosophila immune deficiency (IMD) pathway...,2008,FASEB journal : official publication of the Fe...,4.80,Schmidt RL,Tang AH,Department of Biochemistry and Molecular Biolo...
397,1884,Avet-Rochex A;Perrin J;Bergeret E;Fauvarque MO,Rac2 is a major actor of Drosophila resistance...,17903178,12,10,Pathogen recognition and engulfment by phagocy...,2007,Genes to cells : devoted to molecular &amp; ce...,2.10,Avet-Rochex A,Fauvarque MO,"Commissariat à l'Energie Atomique, DSV, iRTSV,..."
398,2041,Tanji T;Ohashi-Kobayashi A;Natori S,Participation of a galactose-specific C-type l...,16475980,396,1,A galactose-specific C-type lectin has been pu...,2006,The Biochemical journal,4.10,Tanji T,Natori S,"Department of Cell Biochemistry, Graduate Scho..."


In [88]:
authors = clean_df(dfs["authors"].copy())
authors
career_stage = dfs["career_stages"].rename(columns={"id": "career_stage_id"})
authors = authors.merge(career_stage, on="career_stage_id", how="left", suffixes=('', '_career_stage'))
expertise_level = dfs["expertise_levels"].rename(columns={"id": "expertise_level_id"})
authors = authors.merge(expertise_level, on="expertise_level_id", how="left", suffixes=('', '_expertise_level'))

authors = clean_df(authors.drop(columns=["career_stage_id", 
                                        "expertise_level_id"])).rename(columns={"id": "author_id"})
authors = authors.merge(articles, on="article_id", how="left", suffixes=('', '_article'))
authors.sort_values(by=["article_id", "author_id", ])

,author_id,article_id,name,sex,leading_author,first_author,name_career_stage,name_expertise_level,authors_txt,title,pmid,volume,issue,abstract,year,journal_name,impact_factor,first_author_extracted,last_author_extracted,primary_affiliation
752,775,1403,Novakova M,0,False,True,PhD,Experienced,Novakova M;Dolezal T,Expression of Drosophila adenosine deaminase i...,21412432,6,3,Extra-cellular adenosine is an important regul...,2011,PloS one,3.7,Novakova M,Dolezal T,"Department of Molecular Biology, Faculty of Sc..."
753,776,1403,Dolezal T,1,True,False,Junior PI,Experienced,Novakova M;Dolezal T,Expression of Drosophila adenosine deaminase i...,21412432,6,3,Extra-cellular adenosine is an important regul...,2011,PloS one,3.7,Novakova M,Dolezal T,"Department of Molecular Biology, Faculty of Sc..."
720,741,1414,Zhao HW,0,False,True,PhD,Newcomer,Zhao HW;Zhou D;Haddad GG,Antimicrobial peptides increase tolerance to o...,21148307,286,8,It is well appreciated that reactive oxygen sp...,2011,The Journal of biological chemistry,4.8,Zhao HW,Haddad GG,"Division of Respiratory Medicine, Department o..."
721,742,1414,Haddad GG,1,True,False,Junior PI,Newcomer,Zhao HW;Zhou D;Haddad GG,Antimicrobial peptides increase tolerance to o...,21148307,286,8,It is well appreciated that reactive oxygen sp...,2011,The Journal of biological chemistry,4.8,Zhao HW,Haddad GG,"Division of Respiratory Medicine, Department o..."
722,743,1423,Wang Z,1,False,True,PhD,Newcomer,Wang Z;Flax LA;Kemp MM;Linhardt RJ;Baron MJ,Host and pathogen glycosaminoglycan-binding pr...,21078848,79,2,"During group B streptococcal infection, the al...",2011,Infection and immunity,3.1,Wang Z,Baron MJ,"Channing Laboratory, Department of Medicine, B..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
308,329,3461,Hultmark D,1,True,False,Senior PI,Experienced,Williams MJ;Ando I;Hultmark D,Drosophila melanogaster Rac2 is necessary for ...,16098145,10,8,It has been reported that during Drosophila em...,2005,Genes to cells : devoted to molecular &amp; ce...,2.1,Williams MJ,Hultmark D,"Umeå Centre for Molecular Pathogenesis, Umeå U..."
197,204,3462,Khush RS,1,False,True,Post-doc,Experienced,Khush RS;Cornwell WD;Uram JN;Lemaitre B,A ubiquitin-proteasome pathway represses the D...,12401167,12,20,The inducible production of antimicrobial pept...,2002,Current biology : CB,9.2,Khush RS,Lemaitre B,Centre de G&#xe9;n&#xe9;tique Mol&#xe9;culaire...
220,205,3462,Lemaitre B,1,True,False,Junior PI,Experienced,Khush RS;Cornwell WD;Uram JN;Lemaitre B,A ubiquitin-proteasome pathway represses the D...,12401167,12,20,The inducible production of antimicrobial pept...,2002,Current biology : CB,9.2,Khush RS,Lemaitre B,Centre de G&#xe9;n&#xe9;tique Mol&#xe9;culaire...
609,650,3463,Nam HJ,1,False,True,PhD,Newcomer,Nam HJ;Jang IH;Asano T;Lee WJ,Involvement of pro-phenoloxidase 3 in lamelloc...,18852525,26,6,"Phenoloxidase (PO), a melanin-forming enzyme a...",2008,Molecules and cells,3.8,Nam HJ,Lee WJ,"Division of Life and Pharmaceutical Science, D..."


In [89]:
authors.to_csv('preprocessed_data/authors_db.csv', index=False)

## 4. Preprocess the author files

Last files version:
- **0: First** `input_data/2025-03-28/2025-03-27_dropbox_6_ReproSci/0_last version/2025_Last version_March 19th/0_First author cleaned_March 18th.xlsx`
  - Pour les statistiques sur ceux qui deviennent PI, Il ne faut pas prendre en compte ceux qui sont surlignés en bleu car ils sont des premiers auteurs qui sont déjà PI. Normalement ce fichier est bon.  
Avec un peu d’approximation, je trouve que sur 291 first authors: 107 become PI et il y  16  ?? (indéterminés)et 13 qui sont déjà PI en bleu :  ce qui fait 107/262=>40,8% (262=291-16-13)  
- **2: Both** `input_data/2025-03-28/2025-03-27_dropbox_6_ReproSci/0_last version/2025_Last version_March 19th/2_2025_March 9th_stats_author.xlsx`
  - obtenu depuis le site
  -  tu trouveras les last avec de nouveau critères  pour historical (tradition), continuty, first and last.  (pour junior et senior il faut voir par articles dans la database)  
  - Feuille 1 : Parmi les first author (1er feuille) :  
    - Zhenting Zhang et Zhaolin Zhang  sont fusionnés en Zhang Z  
    - Zhipeng Wang et Zhi Wang sont fusionné en Wang Z  
    - Hedengren M et Hedengren M et Hedengren-Olcott M ne sont pas fusionnés
  - Feuille 3: pour les last authors:
    - Il faut fusionner Schneider DS et Schneider D
    - Historical lab (tradition) or not : il faut pendre le colonne C et comparer 1 (historical) et 0 (non historical) mais ne pas prendre en compte 2.
    - La colonne E first and Last : compte les chercheurs qui apparaissent comme first author  et last authors. Il faudrait comparer 1 (last author qui ont été first author dans une autre labo mais pas PI)  à  0 (last author withouth being first author)  sans prendre en compte les 2 (first author who were already PI when first author).
  
- **3:citations** `input_data/2025-03-28/2025-03-27_dropbox_6_ReproSci/0_last version/2025_Last version_March 19th/3_2025_March 9th_citation_counts.xlsm`
  -  il s’agit des citations : je ne l’ai pas travaillé.  
 - **4:first&last** `input_data/2025-03-28/2025-03-27_dropbox_6_ReproSci/0_last version/2025_Last version_March 19th/4_working first and last.xlsx`
    -   ils’agit d’une liste des auteurs qui sont first and last and mais ne pas prendre en compte ceux qui sont PI.  

 It seems that
 - sex -> FH
 - PhD Post-doc -> FH
 - Become a Pi -> FH
 - current job -> FH
 - MD -> FH
 - Affiliation -> Both
 - Country -> Both
 - Ivy League -> Both

In [11]:
# read an xlx file in pandas
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import  preprocess_utils

stat_author_fn = "input_data/2025-03-28/2025-03-27_dropbox_6_ReproSci/0_last version/2025_Last version_March 19th/2_2025_March 9th_stats_author.xlsx"

claims_df = pd.read_csv('preprocessed_data/claims_truncated_for_llm.csv')
major_claims_df = claims_df[claims_df['assertion_type'] == 'major_claim']

# Read Author info, which contains all the pairs
paper_auth_pairs = pd.read_excel('input_data/2025-02-14_last_xlsx/1_Triage_Last author.xlsx', sheet_name='Tri sans les doublons')
# Drop all columns with 'Unnamed' in the name
paper_auth_pairs = paper_auth_pairs.drop(columns=[col for col in paper_auth_pairs.columns if 'Unnamed' in col]).drop(columns=['Source'])
# rename RIZKI MT	to Rizki TM in column last author
paper_auth_pairs['last author'] = paper_auth_pairs['last author'].replace({'Rizki MT': 'Rizki TM','RIZKI MT': 'Rizki TM' })
paper_auth_pairs.to_csv('input_data/2025-02-14_last_xlsx/1_Triage_Last author.csv', index=False)

first_authors_claims = pd.read_excel(stat_author_fn, sheet_name='First')
leading_authors_claims = pd.read_excel(stat_author_fn, sheet_name='Leading')
leading_authors_claims["Authorship"]= "Leading"
first_authors_claims["Authorship"]= "First"

authors_claims = pd.concat([leading_authors_claims, first_authors_claims])
authors_claims['Sex'] = authors_claims['Sex'].map({1: 'Male', 0: 'Female'})
authors_claims = authors_claims.drop(columns=[col for col in authors_claims.columns if '%' in col])
authors_claims = authors_claims.drop(columns=[col for col in authors_claims.columns if 'Unnamed' in col])

authors_claims = authors_claims.rename(columns={'Conituinity': 'Continuity', 
                                                "Partially verified":"Partially Verified", 
                                                "*Historical lab after 1998":"Historical lab after 1998"
                                                })

authors_claims['Historical lab'] = authors_claims['Historical lab'].astype('boolean')
authors_claims['Continuity'] = authors_claims['Continuity'].astype('boolean')
authors_claims = authors_claims.dropna(subset=['Name'])
# drop  Schneider DS+D from names as  Schneider DS and  Schneider D are also there in separate rows
authors_claims = authors_claims[~(authors_claims['Name'] == 'Schneider DS+D')]

authors_claims.to_csv('input_data/2025-02-14_last_xlsx/stats_author.csv', index=False)

### A. Last authors

In [12]:
paper_auth_pairs

,last author,first author,Sex,PhD Post-doc,Become a Pi,current job,MD,Affiliation,Country,Ivy league
0,Agaisse H,Derré,0,Post-doc,1,PI,0,Yale University,USA,1
1,Aguilera RJ,Seong CS,1,PhD,0,Academia,0,The University of Texas,USA,0
2,Aigaki T,Tsuda M,1,PhD,1,Admin,0,Tokyo Metropolitan University,Japan,0
3,Ando,Markus,1,PhD,0,Facility leader,0,"Hungarian Academy of Sciences, Szeged",Hungary,0
4,Ando,Rus F,0,PhD,0,??,0,"Hungarian Academy of Sciences, Szeged",Hungary,0
...,...,...,...,...,...,...,...,...,...,...
315,Yu XQ,Ao J,??,PhD,??,Senior Staff,0,niversity of Missouri-Kansas City,USA,0
316,Zhu S,Yuan Y,1,PhD,0,Senior Staff,0,"Institute of Zoology, Chinese Academy of Sciences",China,0
317,Zhu S,Tian C,0,PhD,1,PI,0,"Institute of Zoology, Chinese Academy of Sciences",China,0
318,Zhu S,Zhang Z,0,PhD,0,Academia,0,"Institute of Zoology, Chinese Academy of Sciences",China,0


In [13]:
paper_auth_pairs_LH = paper_auth_pairs[["last author", "Affiliation", "Country", "Ivy league"]]
paper_auth_pairs_LH = preprocess_utils.deduplicate_by(paper_auth_pairs_LH, "last author").copy()
claims_LH = authors_claims[authors_claims['Authorship'] == 'Leading'].copy()

paper_auth_pairs_LH.loc[:, 'lh_proc'] = paper_auth_pairs_LH['last author'].str.lower()
paper_auth_pairs_LH.loc[:, 'lh_proc'] = (paper_auth_pairs_LH['lh_proc']
    .str.normalize('NFKD')
    .str.encode('ascii', errors='ignore')
    .str.decode('utf-8'))
#paper_auth_pairs_LH.loc[:, 'lh_proc'] = paper_auth_pairs_LH['lh_proc'].str.replace('rizki mt', 'rizki tm')

# For claims_LH
# drop the row where Name that is NaN 
claims_LH.loc[:, 'lh_proc'] = claims_LH['Name'].str.lower()
claims_LH.loc[:, 'lh_proc'] = (claims_LH['lh_proc']
    .str.normalize('NFKD')
    .str.encode('ascii', errors='ignore')
    .str.decode('utf-8'))
claims_LH.loc[:, 'lh_proc'] = claims_LH['lh_proc'].str.replace('ando i', 'ando')

In [14]:
# check for duplicate lh_proc in paper_auth_pairs_LH
paper_auth_pairs_LH[paper_auth_pairs_LH.duplicated('lh_proc', keep=False)]
print(len(paper_auth_pairs_LH[paper_auth_pairs_LH.duplicated('lh_proc', keep=False)]), "duplicates in paper_auth_pairs_LH")
# check for duplicate lh_proc in claims_LH
print(len(claims_LH[claims_LH.duplicated('lh_proc', keep=False)]), "duplicates in claims_LH")

0 duplicates in paper_auth_pairs_LH
0 duplicates in claims_LH


In [15]:
# Perform the outer merge to see what we are missing
all_LH = pd.merge(claims_LH, paper_auth_pairs_LH, on='lh_proc', how='outer')
print(len(claims_LH), len(paper_auth_pairs_LH), len(all_LH))

157 160 161


In [16]:
unique_pairs = all_LH[["Name", "last author", "lh_proc"]].drop_duplicates().sort_values("lh_proc", ascending=True)
print(" ONLY in Pairs               only in Claims        matching key ")
print("-"*70)
for i in range(0, len(unique_pairs)):
    if pd.isna(unique_pairs.iloc[i]['last author']) or pd.isna(unique_pairs.iloc[i]['Name']):
        print('💥 ', end='')
        print(f"{unique_pairs.iloc[i]['last author']:<20} vs  {unique_pairs.iloc[i]['Name']:<20} --> {unique_pairs.iloc[i]['lh_proc']:<20}")
    #else:
    #    print(f"   {unique_pairs.iloc[i]['last author']:<20} vs  {unique_pairs.iloc[i]['Name']:<20} --> {unique_pairs.iloc[i]['lh_proc']:<20}")

 ONLY in Pairs               only in Claims        matching key 
----------------------------------------------------------------------
💥 Bellotti RA          vs  nan                  --> bellotti ra         
💥 nan                  vs  Nappi AJ             --> nappi aj            
💥 Shahabuddin M        vs  nan                  --> shahabuddin m       
💥 Shirasu-Hiza MM      vs  nan                  --> shirasu-hiza mm     
💥 Silvers MJ           vs  nan                  --> silvers mj          


In [17]:
all_LH_inner = pd.merge(claims_LH, paper_auth_pairs_LH, on='lh_proc', how='inner').sort_values("lh_proc", ascending=True)
#all_LH_inner.drop(columns=['lh_proc', 'last author', 'Authorship'], inplace=True)
print(len(all_LH_inner))
all_LH_inner.to_csv('preprocessed_data/LH_inner.csv', index=False)
all_LH_inner

156


,Name,Historical lab,Historical lab after 1998,Continuity,F and L,Sex,Articles,Major claims,Unchallenged,Verified,...,Mixed,Challenged,Start lab,Finish,Authorship,lh_proc,last author,Affiliation,Country,Ivy league
63,Agaisse H,False,0.0,False,1.0,Male,1.0,3.0,2.0,1.0,...,0.0,0.0,NaN,NaN,Leading,agaisse h,Agaisse H,Yale University,USA,1
107,Aguilera RJ,False,0.0,False,0.0,Male,1.0,2.0,0.0,1.0,...,0.0,1.0,NaN,NaN,Leading,aguilera rj,Aguilera RJ,The University of Texas,USA,0
64,Aigaki T,False,0.0,False,0.0,Male,1.0,3.0,2.0,1.0,...,0.0,0.0,NaN,NaN,Leading,aigaki t,Aigaki T,Tokyo Metropolitan University,Japan,0
6,Anderson KV,False,0.0,False,0.0,Female,8.0,20.0,7.0,11.0,...,0.0,2.0,1997,2010,Leading,anderson kv,Anderson KV,Memorial Sloan-Kettering Cancer Center and the...,USA,1
16,Andó I,False,0.0,True,0.0,Male,6.0,12.0,1.0,7.0,...,0.0,0.0,1999,still active,Leading,ando,Ando,"Hungarian Academy of Sciences, Szeged",Hungary,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
104,Xu T,False,0.0,False,0.0,Male,1.0,3.0,0.0,3.0,...,0.0,0.0,NaN,NaN,Leading,xu t,Xu T,Yale University School of Medicine,USA,0
146,Yamaguchi M,False,0.0,False,0.0,Male,1.0,2.0,1.0,1.0,...,0.0,0.0,NaN,NaN,Leading,yamaguchi m,Yamaguchi M,Aichi Cancer Center Research Institute,Japan,0
105,Yoo MA,False,0.0,False,0.0,Female,1.0,3.0,0.0,2.0,...,0.0,0.0,NaN,NaN,Leading,yoo ma,Yoo MA,Pusan National University,Korea,0
106,Yu XQ,False,0.0,False,0.0,Male,1.0,3.0,1.0,2.0,...,0.0,0.0,NaN,NaN,Leading,yu xq,Yu XQ,niversity of Missouri-Kansas City,USA,0


### B. First authors: TODO

In [18]:
paper_auth_pairs

,last author,first author,Sex,PhD Post-doc,Become a Pi,current job,MD,Affiliation,Country,Ivy league
0,Agaisse H,Derré,0,Post-doc,1,PI,0,Yale University,USA,1
1,Aguilera RJ,Seong CS,1,PhD,0,Academia,0,The University of Texas,USA,0
2,Aigaki T,Tsuda M,1,PhD,1,Admin,0,Tokyo Metropolitan University,Japan,0
3,Ando,Markus,1,PhD,0,Facility leader,0,"Hungarian Academy of Sciences, Szeged",Hungary,0
4,Ando,Rus F,0,PhD,0,??,0,"Hungarian Academy of Sciences, Szeged",Hungary,0
...,...,...,...,...,...,...,...,...,...,...
315,Yu XQ,Ao J,??,PhD,??,Senior Staff,0,niversity of Missouri-Kansas City,USA,0
316,Zhu S,Yuan Y,1,PhD,0,Senior Staff,0,"Institute of Zoology, Chinese Academy of Sciences",China,0
317,Zhu S,Tian C,0,PhD,1,PI,0,"Institute of Zoology, Chinese Academy of Sciences",China,0
318,Zhu S,Zhang Z,0,PhD,0,Academia,0,"Institute of Zoology, Chinese Academy of Sciences",China,0


In [46]:
paper_auth_pairs

,last author,first author,Sex,PhD Post-doc,Become a Pi,current job,MD,Affiliation,Country,Ivy league
0,Agaisse H,Derré,0,Post-doc,1,PI,0,Yale University,USA,1
1,Aguilera RJ,Seong CS,1,PhD,0,Academia,0,The University of Texas,USA,0
2,Aigaki T,Tsuda M,1,PhD,1,Admin,0,Tokyo Metropolitan University,Japan,0
3,Ando,Markus,1,PhD,0,Facility leader,0,"Hungarian Academy of Sciences, Szeged",Hungary,0
4,Ando,Rus F,0,PhD,0,??,0,"Hungarian Academy of Sciences, Szeged",Hungary,0
...,...,...,...,...,...,...,...,...,...,...
315,Yu XQ,Ao J,??,PhD,??,Senior Staff,0,niversity of Missouri-Kansas City,USA,0
316,Zhu S,Yuan Y,1,PhD,0,Senior Staff,0,"Institute of Zoology, Chinese Academy of Sciences",China,0
317,Zhu S,Tian C,0,PhD,1,PI,0,"Institute of Zoology, Chinese Academy of Sciences",China,0
318,Zhu S,Zhang Z,0,PhD,0,Academia,0,"Institute of Zoology, Chinese Academy of Sciences",China,0


In [47]:
paper_auth_pairs_FH = paper_auth_pairs.drop(columns=['last author']).sort_values("first author", ascending=True)
paper_auth_pairs_FH = preprocess_utils.deduplicate_by(paper_auth_pairs_FH, "first author")

In [48]:
claims_FH[claims_FH.duplicated('fh_proc', keep=False)]

,Name,Historical lab,Historical lab after 1998,Continuity,F and L,Sex,Articles,Major claims,Unchallenged,Verified,Partially Verified,Mixed,Challenged,Start lab,Finish,Authorship,fh_proc
51,Kim YS,<NA>,NaN,<NA>,NaN,Male,2.0,5.0,0.0,2.0,1.0,0.0,2.0,NaN,NaN,First,kim ys
122,Hedengren M,<NA>,NaN,<NA>,NaN,Female,1.0,3.0,0.0,2.0,1.0,0.0,0.0,NaN,NaN,First,hedengren m
135,Kim YS,<NA>,NaN,<NA>,NaN,Female,1.0,3.0,0.0,2.0,1.0,0.0,0.0,NaN,NaN,First,kim ys
216,Hedengren M,<NA>,NaN,<NA>,NaN,Male,1.0,2.0,0.0,2.0,0.0,0.0,0.0,NaN,NaN,First,hedengren m


In [ ]:
claims_FH = authors_claims[authors_claims['Authorship'] == 'First'].copy()

# create merge columns: lowercased and stripped of accents
paper_auth_pairs_FH['fh_proc'] = paper_auth_pairs_FH['first author'].str.lower()
paper_auth_pairs_FH['fh_proc'] = paper_auth_pairs_FH['fh_proc'].str.normalize('NFKD').str.encode('ascii', errors='ignore').str.decode('utf-8')
paper_auth_pairs_FH['fh_proc'] = paper_auth_pairs_FH['fh_proc'].str.replace('derre', 'derre i')
paper_auth_pairs_FH['fh_proc'] = paper_auth_pairs_FH['fh_proc'].str.replace('markus', 'markus r')
claims_FH['fh_proc'] = claims_FH['Name'].str.lower()
claims_FH['fh_proc'] = claims_FH['fh_proc'].str.normalize('NFKD').str.encode('ascii', errors='ignore').str.decode('utf-8')


# check for duplicate lh_proc in paper_auth_pairs_LH
print(len(paper_auth_pairs_FH[paper_auth_pairs_FH.duplicated('fh_proc', keep=False)]), "duplicates in paper_auth_pairs_LH")
# check for duplicate lh_proc in claims_LH
print(len(claims_FH[claims_FH.duplicated('fh_proc', keep=False)]), "duplicates in claims_LH")


all_FH = pd.merge(claims_FH, paper_auth_pairs_FH, on='fh_proc', how='outer')
print(len(claims_FH), len(paper_auth_pairs_FH), len(all_FH))

print(" ONLY in Pairs               only in Claims        matching key ")
print("-"*70)
unique_pairs = all_FH[["Name", "first author", "fh_proc"]].drop_duplicates().sort_values("fh_proc", ascending=True)
for i in range(0, len(unique_pairs)):
    if pd.isna(unique_pairs.iloc[i]['first author']) or pd.isna(unique_pairs.iloc[i]['Name']):
        print('💥 ', end='')
        print(f" {unique_pairs.iloc[i]['first author']:<20}  {unique_pairs.iloc[i]['Name']:<20} --> {unique_pairs.iloc[i]['fh_proc']:<20}")

0 duplicates in paper_auth_pairs_LH
4 duplicates in claims_LH
292 291 294
 ONLY in Pairs               only in Claims        matching key 
----------------------------------------------------------------------
💥  nan                   Hedengren-Olcott M   --> hedengren-olcott m  
💥  RIZKI MT              nan                  --> rizki mt            
💥  Schneider D           nan                  --> schneider d         


In [ ]:
all_FH_inner = pd.merge(claims_FH, paper_auth_pairs_FH, on='fh_proc', how='inner').sort_values("fh_proc", ascending=True)
all_FH_inner.drop(columns=['Continuity', 'Historical lab after 1998', 'Historical lab', "Sex_y"], inplace=True)
# rename Sex_x to Sex
all_FH_inner = all_FH_inner.rename(columns={'Sex_x': 'Sex'})
print(len(all_FH_inner))
all_FH_inner.to_csv('preprocessed_data/FH_inner.csv', index=False)
all_FH_inner

291


,Name,F and L,Sex,Articles,Major claims,Unchallenged,Verified,Partially Verified,Mixed,Challenged,...,Authorship,fh_proc,first author,PhD Post-doc,Become a Pi,current job,MD,Affiliation,Country,Ivy league
80,Abdelsadik A,NaN,Male,1.0,3.0,3.0,0.0,0.0,0.0,0.0,...,First,abdelsadik a,Abdelsadik A,Senior Staff,1,PI,0,Forschungszentrum Borstel,Germany,0
19,Agaisse H,NaN,Male,2.0,6.0,2.0,4.0,0.0,0.0,0.0,...,First,agaisse h,Agaisse H,Post-doc,1,PI,0,Harvard Medical School,USA,1
196,Aggarwal K,NaN,Female,1.0,2.0,0.0,2.0,0.0,0.0,0.0,...,First,aggarwal k,Aggarwal K,PhD,0,Industry,0,University of Massachusetts Medical School,USA,0
81,Ao J,NaN,Male,1.0,3.0,1.0,2.0,0.0,0.0,0.0,...,First,ao j,Ao J,PhD,??,Senior Staff,0,niversity of Missouri-Kansas City,USA,0
20,Apidianakis Y,NaN,Male,3.0,6.0,4.0,2.0,0.0,0.0,0.0,...,First,apidianakis y,Apidianakis Y,Post-doc,1,PI,0,Harvard Medical School and Massachusetts Gener...,USA,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
194,Zettervall CJ,NaN,Male,1.0,3.0,0.0,3.0,0.0,0.0,0.0,...,First,zettervall cj,Zettervall CJ,PhD,0,Industry,0,University of Stockholm,Sweden,0
273,Zhang Z,NaN,Female,2.0,2.0,1.0,1.0,0.0,0.0,0.0,...,First,zhang z,Zhang Z,PhD,0,Academia,0,"Institute of Zoology, Chinese Academy of Sciences",China,0
195,Zhao HW,NaN,Female,1.0,3.0,2.0,1.0,0.0,0.0,0.0,...,First,zhao hw,Zhao HW,PhD,0,??,0,University of California San Diego,USA,1
290,Zhou Z,NaN,Male,1.0,1.0,0.0,1.0,0.0,0.0,0.0,...,First,zhou z,Zhou Z,PhD,??,??,0,University of Houston,USA,0
